# Preprocessing training data

This is the continuation of my previous article. In this article, I will apply a preprocessing method on the dataset that was presented in the previous article (*Keras1.ipynb*), followed by the training of the deep learning networks. At the end, testing results will be presented.

It should be noted that the dataset that was presented in the previous article had uneven distribution of true and false cases. In particular, the dataset had 268 true cases and 500 false cases of diabetese. Details on the dataset can be found in the following link: 
https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names

The deep learning model trained using such dataset can lead to the biasness in the trained network outcome because more true cases are fed to the network than false cases. Such issues has been discussed in the following article by Satya Mallick:

https://www.learnopencv.com/bias-variance-tradeoff-in-machine-learning/

To avoid it, the dataset should have equal numbers of true and false cases, respectively. Therefore let us create new data that has the property. Furthermore, the training and testing of the model should be done in different sets. Therefore, the data set will be divided into 4 such sets, and one set will be used to train, whereas the remainings are used for testing the model.

The preprocessing steps can be explained by the following figure
![alt text](preprocessingProcess.png)

In [0]:
!pip install --upgrade -q gspread

from google.colab import auth
auth.authenticate_user()


import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('pima-indians-diabetes').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()



# Convert to a DataFrame and render.
import pandas as pd
import numpy  as np
dataset =pd.DataFrame.from_records(rows)

#Converting to numpy arrays for X and Y
WholeData=dataset.values[:,:] 
true_case =np.empty([1,9])
false_case =np.empty([1,9])

#creating two dataset for true and false case
for row in WholeData:
   if(row[8]=='1'):
    true_case= np.vstack((true_case,row))
   else:
    false_case= np.vstack((false_case,row))
    
#deleting the first row as it contains garbage due to empty
false_case = false_case[1:-1,:] 
true_case =true_case[1:-1,:]

#creating 4 different sets from true_case and false case
dataSet =[]
for i in range(4):
  temp_set = np.empty([1,9])
  for j in range(66):
    temp_set = np.vstack((temp_set,true_case[i*67+j,:]))
    temp_set = np.vstack((temp_set,false_case[i*67+j,:]))
  temp_set = temp_set[1:-1,:]
  dataSet.append(temp_set)

